# ICS 471-Deep Learning using Keras

We will cover how to develop real-world deep learning models using Keras.

First you need to have keras installed on your machine (if you're using colab, it is already installed).



## Data loading

Keras models accept three types of inputs:

- **NumPy arrays**, just like Scikit-Learn and many other Python-based libraries. This
 is a good option if your data fits in memory.
- **[TensorFlow `Dataset` objects](https://www.tensorflow.org/guide/data)**. This is a
high-performance option that is more suitable for datasets that do not fit in memory
 and that are streamed from disk or from a distributed filesystem.
- **Python generators** that yield batches of data (such as custom subclasses of
the `keras.utils.Sequence` class).

Before you start training a model, you will need to make your data available as one of
these formats. If you have a large dataset and you are training on GPU(s), consider
using `Dataset` objects, since they will take care of performance-critical details,
 such as:

- Asynchronously preprocessing your data on CPU while your GPU is busy, and buffering
 it into a queue.
- Prefetching data on GPU memory so it's immediately available when the GPU has
 finished processing the previous batch, so you can reach full GPU utilization.

Keras features a range of utilities to help you turn raw data on disk into a `Dataset`:

- `tf.keras.preprocessing.image_dataset_from_directory` turns image files sorted into
 class-specific folders into a labeled dataset of image tensors.
- `tf.keras.preprocessing.text_dataset_from_directory` does the same for text files.

In addition, the TensorFlow `tf.data` includes other similar utilities, such as
`tf.data.experimental.make_csv_dataset` to load structured data from CSV files.

**Example: obtaining a labeled dataset from image files on disk**

Supposed you have image files sorted by class in different folders, like this:

```
main_directory/
...class_a/
......a_image_1.jpg
......a_image_2.jpg
...class_b/
......b_image_1.jpg
......b_image_2.jpg
```

Then you can do:

```python
# Create a dataset.
dataset = keras.preprocessing.image_dataset_from_directory(
  'path/to/main_directory', batch_size=64, image_size=(200, 200))

# For demonstration, iterate over the batches yielded by the dataset.
for data, labels in dataset:
   print(data.shape)  # (64, 200, 200, 3)
   print(data.dtype)  # float32
   print(labels.shape)  # (64,)
   print(labels.dtype)  # int32
```



**Example: obtaining a labeled dataset from text files on disk**

Likewise for text: if you have `.txt` documents sorted by class in different folders,
 you can do:

```python
dataset = keras.preprocessing.text_dataset_from_directory(
  'path/to/main_directory', batch_size=64)

# For demonstration, iterate over the batches yielded by the dataset.
for data, labels in dataset:
   print(data.shape)  # (64,)
   print(data.dtype)  # string
   print(labels.shape)  # (64,)
   print(labels.dtype)  # int32
```




### Data available in Keras 
If the data is available within the datasets provided by keras, you can load it directly using load_data() as shown below:

In [1]:
# load data
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print(X_train.shape)
print(y_train.shape)

11490434/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)


In [2]:
# reshape to be [samples][width][height][channels]
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1)).astype('float32')
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1)).astype('float32')
print(X_train.shape)
print(X_test.shape)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


## Data preprocessing with Keras

Once your data is in the form of string/int/float NumpPy arrays, or a `Dataset` object
 (or Python generator) that yields batches of string/int/float tensors,
it is time to **preprocess** the data. This can mean:

- Tokenization of string data, followed by token indexing.
- Feature normalization.
- Rescaling the data to small values (in general, input values to a neural network
should be close to zero -- typically we expect either data with zero-mean and
 unit-variance, or data in the `[0, 1]` range.

### How to do preprocessing in Keras?
We can do one of the following:

1.   Preprocess the data before fitting it to the model, or
2.   Preprocess the data within the model (next lecture)

In this lecture, we will preprocess the data before fitting it to the mode. This can be done in two different ways:

1.   Do preprocessing manually
2.   Use objects to do preprocessing 


**Example: Preprocessing the data manually and using objects**


In [3]:
# Method 01: normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

'''
#### Method 02: Do preprocessing using objects: 
import numpy as np
from keras.layers import Rescaling
scaler = Rescaling(scale=1.0 / 255)
 
X_train = scaler(X_train)
X_test = scaler(X_test)
'''

'\n#### Method 02: Do preprocessing using objects: \nimport numpy as np\nfrom keras.layers import Rescaling\nscaler = Rescaling(scale=1.0 / 255)\n \nX_train = scaler(X_train)\nX_test = scaler(X_test)\n'

## Building models with the Keras Functional API

Now, we will build our model using Keras API. We need first to have a data for validation. To do this, we will split the train data (X_train) into train (70%) and validation (30%) data. 

In [4]:
#  split train data into train and val data
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.30, random_state=4, stratify=y_train)
print(X_train.shape)
print(X_val.shape)

(42000, 28, 28, 1)
(18000, 28, 28, 1)


In [5]:
print(y_train[0])

3


We need also to convert the labels to one hot encode.

In [6]:
# one hot encode outputs
from keras.utils import np_utils

y_train = np_utils.to_categorical(y_train)
y_val = np_utils.to_categorical(y_val)
y_test = np_utils.to_categorical(y_test)
num_classes = 10
print("Number of classes: ", num_classes)

Number of classes:  10


In [7]:
print(y_train[0])

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


### Build the model

We can create a NN layer using the different layer types available in keras.layers package. For example, to create a fully connected layer with 16 neurons output.

```python
dense = keras.layers.Dense(units=16)
```

There are two ways to build models in Keras:

1.   Sequential
2.   Functional API

The most common and most powerful way to build Keras models is the Functional API. In the following code, both methods will be covered. To
build models, you start by specifying the shape (and
optionally the dtype) of your inputs. If any dimension of your input can vary, you can
specify it as `None`. For instance, an input for 200x200 RGB image would have shape
`(200, 200, 3)`, but an input for RGB images of any size would have shape `(None,
 None, 3)`.


In [8]:
# define the CNN model using Sequential method
from tensorflow import keras
from keras.models import Sequential

from keras.layers import Dense, Dropout,Flatten 
from keras.layers.convolutional import Conv2D, MaxPooling2D


def build_model():
	# create model
	model = Sequential()
	model.add(Conv2D(30, (5, 5), input_shape=(28, 28, 1), activation='relu'))
	model.add(Conv2D(15, (3, 3), activation='relu'))
	model.add(MaxPooling2D())
	model.add(Dropout(0.2))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dense(50, activation='relu'))
	model.add(Dense(num_classes, activation='softmax'))
	return model

'''
# define the CNN model using functional API method
def build_model():
	# create model
  input_data = keras.Input(shape=(28, 28, 1))
  x = Conv2D(30, (5, 5),  activation='relu')(input_data)
  x = MaxPooling2D()(x)
  x = Conv2D(15, (3, 3), activation='relu')(x)
  x = MaxPooling2D()(x)
  x = Dropout(0.2)(x)
  x = Flatten()(x)
  x = Dense(128, activation='relu')(x)
  x = Dense(50, activation='relu')(x)
  output = Dense(num_classes, activation='softmax')(x)
  model = keras.Model(inputs=input_data, outputs=output)
  return model
'''

"\n# define the CNN model using functional API method\ndef build_model():\n\t# create model\n  input_data = keras.Input(shape=(28, 28, 1))\n  x = Conv2D(30, (5, 5),  activation='relu')(input_data)\n  x = MaxPooling2D()(x)\n  x = Conv2D(15, (3, 3), activation='relu')(x)\n  x = MaxPooling2D()(x)\n  x = Dropout(0.2)(x)\n  x = Flatten()(x)\n  x = Dense(128, activation='relu')(x)\n  x = Dense(50, activation='relu')(x)\n  output = Dense(num_classes, activation='softmax')(x)\n  model = keras.Model(inputs=input_data, outputs=output)\n  return model\n"

Now, we will do the following:


*   Build the model
*   Print the model summary
*   Compile the model to define optimizer, loss function, and evaluation metric(s).
*   Start training the model by fitting it with the data 







In [9]:
# build the model
model = build_model()



In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 30)        780       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 15)        4065      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 11, 15)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 11, 11, 15)        0         
                                                                 
 flatten (Flatten)           (None, 1815)              0         
                                                                 
 dense (Dense)               (None, 128)               232448    
                                                        

### Training models with `fit()`

At this point, you know:

- How to prepare your data (e.g. as a NumPy array or a `tf.data.Dataset` object)
- How to build a model that will process your data

The next step is to train your model on your data. The `Model` class features a
built-in training loop, the `fit()` method. It accepts `Dataset` objects, Python
 generators that yield batches of data, or NumPy arrays.

Before you can call `fit()`, you need to specify an optimizer and a loss function . This is the `compile()` step:

```python
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.CategoricalCrossentropy())
```

Loss and optimizer can be specified via their string identifiers (in this case
their default constructor argument values are used):


```python
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
```

Once your model is compiled, you can start "fitting" the model to the data.
Here's what fitting a model looks like with NumPy data:

```python
model.fit(numpy_array_of_samples, numpy_array_of_labels,validation_data=(X_val, y_val),
          batch_size=32, epochs=10)
```

Besides the data, you have to specify two key parameters: the `batch_size` and
the number of epochs (iterations on the data). Here our data will get sliced on batches
 of 32 samples, and the model will iterate 10 times over the data during training.

Here's what fitting a model looks like with a dataset:

```python
model.fit(dataset_of_samples_and_labels, epochs=10)
```

Since the data yielded by a dataset is expected to be already batched, you don't need to
 specify the batch size here.



In [11]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [14]:
# Fit the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=3)


Epoch 1/3
420/420 [==============================] - 63s 150ms/step - loss: 0.0872 - accuracy: 0.9737 - val_loss: 0.0659 - val_accuracy: 0.9807
Epoch 2/3
420/420 [==============================] - 62s 149ms/step - loss: 0.0565 - accuracy: 0.9825 - val_loss: 0.0530 - val_accuracy: 0.9843
Epoch 3/3
420/420 [==============================] - 60s 143ms/step - loss: 0.0411 - accuracy: 0.9869 - val_loss: 0.0525 - val_accuracy: 0.9852


The `fit()` call returns a "history" object which records what happened over the course
of training. The `history.history` dict contains per-epoch timeseries of metrics
values (here we have only one metric, the loss, and one epoch, so we only get a single
 scalar):

In [ ]:
print(history.history)

{'loss': [0.5173812508583069, 0.12352829426527023, 0.08717623353004456], 'accuracy': [0.836904764175415, 0.963523805141449, 0.9723571538925171], 'val_loss': [0.12367802113294601, 0.0756782814860344, 0.06525196135044098], 'val_accuracy': [0.9647777676582336, 0.9772777557373047, 0.980555534362793]}


### Using callbacks for checkpointing (and more)

If training goes on for more than a few minutes, it's important to save your model at
 regular intervals during training. You can then use your saved models
to restart training in case your training process crashes (this is important for
multi-worker distributed training, since with many workers at least one of them is
 bound to fail at some point).

An important feature of Keras is **callbacks**, configured in `fit()`. Callbacks are
 objects that get called by the model at different point during training, in particular:

- At the beginning and end of each batch
- At the beginning and end of each epoch

Callbacks are a way to make model trainable entirely scriptable.

You can use callbacks to periodically save your model. Here's a simple example: a
 `ModelCheckpoint` callback
configured to save the model at the end of every epoch. The filename will include the
 current epoch.

```python
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='path/to/my/model_{epoch}',
        save_freq='epoch')
]
model.fit(dataset, epochs=2, callbacks=callbacks)
```

### Monitoring training progress with TensorBoard

Staring at the Keras progress bar isn't the most ergonomic way to monitor how your loss
 and metrics are evolving over time. There's a better solution:
[TensorBoard](https://www.tensorflow.org/tensorboard),
a web application that can display real-time graphs of your metrics (and more).

To use TensorBoard with `fit()`, simply pass a `keras.callbacks.TensorBoard` callback
 specifying the directory where to store TensorBoard logs:


```python
callbacks = [
    keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(dataset, epochs=2, callbacks=callbacks)
```

You can then launch a TensorBoard instance that you can open in your browser to monitor
 the logs getting written to this location:

```
tensorboard --logdir=./logs
```

What's more, you can launch an in-line TensorBoard tab when training models in Jupyter
 / Colab notebooks.
[Here's more information](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks).

### Evaluate the model on the test data
After we trained the model, we will evaluate it on the test data

In [15]:
# Final evaluation of the model
loss, acc = model.evaluate(X_test, y_test)
print("CNN Accuracy: %.2f%%" % (acc*100))

313/313 [==============================] - 5s 14ms/step - loss: 0.0385 - accuracy: 0.9873
CNN Accuracy: 98.73%
